# SPARTA

We have SARS-CoV-2 spike-binding measurements available on 10,000 samples from 500 participants.
We want to:
- Look at what we have
- Pick the top 30 or so and the bottom 30 or so participants, by "antibody response strength"
- Clean up and deidentify metadata for those participants
- Send out for additional testing

In addition to those measurements, we know that participants have different demographics, SARS-CoV-2 infection and vaccination timing, vaccine types, and sampling frequency (we tried to get them monthly, but sometimes that's just how it goes)

## Imports

In [ ]:
import pandas as pd # Excel+, tables and more
import numpy as np # Maths
import matplotlib.pyplot as plt # Setting up figures, axis labels, etc
import seaborn as sns # Scatterplots and stripplots and histograms, oh my
from scipy import stats # T tests, mann whitney u, ordinary least squares linear regression
import sys # for difficulties installing packages
!{sys.executable} -m pip install statsmodels
import statsmodels.api as sm # robust lineear regression
import statsmodels.formula.api as smf # mixed effect models
import sklearn # multilayer perceptrons, support vector machines, random forests


## Python Reference

In [ ]:
# Comments
# Any line starting with a hash is a comment
# In addition to markdown cells (or in a script), you can annotate your code in-line with comments to describe what it does
# In order to change a line to and from a comment, you can use the shortcut Ctrl+/ (or Cmd+/ on Mac)

# Uncomment the bottom line (click on the line to place your cursor in and hit Ctrl+/)
# and run the cell (press the play button in the upper left corner, or Shift+Enter with your cursor in the cell) to generate a random plot

# plt.plot(np.random.randn(100))

In [ ]:
# Seaborn maintains a few datasets useful for example plots. 
# You can look at the first 5 rows of your dataset with the head() function.
# Run this cell to see what's in the penguins dataset. (shift+enter to run)
penguin_data = sns.load_dataset('penguins')
penguin_data.head()

In [ ]:
# You can look at a couple default summary statistics of the dataset with the describe() function.
penguin_data.describe()

Notice that it dropped all the non-numeric columns, and gives you these count statistics. You can also use `describe()` for non-numeric data, but it is not quite as useful.

Still, it does take us to our next operation: grabbing just one column from the table. In pandas, usually a table is called a "DataFrame" and a row or column is called a "Series". If we want to look at just one column, we use square braces [] and single or double quotes containing the column name. To look at just species information, we would call `penguin_data['species']` instead of `penguin_data`. Let's look at the description for just that Series

In [ ]:
penguin_data['species'].describe()

Not bad, but not great. You know that there are 3 values, but not what they are. This could be good if you had thousands of different species, but with just 3 we might want more details about each one. There's a function on Series called `value_counts()` which works well here.

In [ ]:
# Calling `display()` lets you show the output of multiple rows in a single notebook cell.
display(penguin_data['species'].value_counts())
display(penguin_data['island'].value_counts())

Now we have some counts and some basic statistics. We can do a little better and break these penguin features down by both species and island with the `groupby` command.

In [ ]:
display(penguin_data.groupby(['species', 'island']).size())

Looks like only the Adelie are travelers. Let's forget about islands, then, and look at flippers and beaks

In [ ]:
display(penguin_data.drop(columns=['island', 'sex']).groupby(['species']).mean()) # You can't take tghe mean of a string, so we drop the non-numeric columns. Equivalent to:
display(penguin_data.loc[:, ['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].groupby(['species']).mean())

Couple of new operations there:
- `data.drop(columns=['col1', 'col2'])` gives you a copy of your dataframe (table) without col1 and col2
- `data.loc[:, 'col1']` is the same as `data['col1']`, it just grabs that column. Here `:` stands for "every row", so it extends to:
- `data.loc[:, ['col1', 'col2']]` to get the two columns you want. `loc[]` uses column and row names. If I wanted the first two columns, I could use `iloc`
- `data.iloc[:5, :2]` is the first two columns for the first 5 rows
- `data.iloc[-5:, -2:]` is the last two columns for the last 5 rows

`iloc` uses numbers and does not know about names at all. See below for an example. 

In [ ]:
penguin_data.iloc[:5, :2]

You can also filter quite easily any which way with indexes.

In [ ]:
display(penguin_data[penguin_data['species'] == 'Chinstrap'].head()) # chinstrap penguins
species_filter = penguin_data['species'] == 'Adelie'
sex_filter = penguin_data['sex'] == 'Male'
display(penguin_data[species_filter & sex_filter].head()) # male Adelie penguins
size_filter = penguin_data['body_mass_g'] < 3800.
display(penguin_data[species_filter & sex_filter & size_filter].head()) # tiny male Adelie penguins

`==` checks equality. You can compose filters with `&` when you want both to be true, and `|` when only one needs to be true. You can compare Series to individual values, or to each other

In [ ]:
display(penguin_data[penguin_data['bill_depth_mm'] * 2 < penguin_data['bill_length_mm']].head()) # bill depth is less than half the bill length
penguin_data[penguin_data['bill_depth_mm'] * 2 > penguin_data['bill_length_mm']].head() # bill depth is greater than half the bill length

Finally, plots! `sns.relplot` (scatter and line) and `sns.catplot` (strip and box) will help you out here

In [ ]:
sns.relplot(data=penguin_data, x='bill_length_mm', y='bill_depth_mm', hue='species')
plt.show()

In [ ]:
sns.catplot(data=penguin_data, x='species', y='body_mass_g', hue='island', dodge=True, kind='strip')
plt.show()

## Reading in the data and taking a look

We have participant metadata and sample testing results in the `data` directory. Poke around! Make some plots!

In [ ]:
people_data = pd.read_csv('../data/Participant Metadata.csv').set_index('Participant ID') # Reads in a csv from the data directory
display(people_data.head())
# sample_data =  # Uncomment this line and add the code to load the participant data
# display(sample_data.head()) 
# data = sample_data.join(people_data, on='Participant ID') # Adds the participant data to the sample data. Works as is, uncomment with Ctrl+/ to run
# display(data.head()) # Displays the first 5 rows of the participant data

## Picking overperformers and underperformers

Now that you've taken a look, it's onto the next task

### High and low performers
We want to run additional testing on a subset of participants (10 or 20%). How would you pick a subset of participants to run?
- You can answer this with a markdown cell at first if you're not ready to code it.
- How would you do it if you were totally confident in Python?
- What's one quick way that you would be comfortable doing now?

## Exporting deidentified metadata

Once we pick participants and samples, we want to provide the associated metadata to whoever is running the additional testing.
- What information do they need?
- What information can't they have? (emails, for sure)
- What kinds of testing would you want to do? You'll get back WA.1 SARS-CoV-2 neutralization, for sure, but is there anything else that would help?

Once you have an idea for that, make a subset of the datasheet and export it. To make a csv, you take a dataframe `data` and call `data.to_csv('output_filename.csv')`